In [73]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import joblib
import tensorflow as tf
from carla.recourse_methods import GrowingSpheres, Dice, ActionableRecourse, CEM, CausalRecourse, Face
from carla import MLModelCatalog

In [32]:
def display_results(model, factual, counterfactuals):
    pred = model.predict(factual)
    print('Factual class:', pred.round(3)) 
    print('Counterfactuals class:')
    for pred in model.predict(counterfactuals):
        print(pred.round(3))

In [6]:
from carla.data.catalog import CsvCatalog
import json

with open('../data/adult_constraints.json', 'r') as f:
    constraints = json.load(f)

continuous = constraints['continuous']
categorical = constraints['categorical']
immutable = constraints['immutable']
columns_order = constraints['features_order']

dataset = CsvCatalog(file_path="../data/adult_prep.csv",
                    continuous=continuous,
                    categorical=categorical,
                    immutables=immutable,
                    target='income')
                    
dataset.df.head(2)

,age,education.num,capital.gain,capital.loss,hours.per.week,...,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,1.000000,0.533333,0.0,1.0,0.397959,...,0.0,0.0,0.0,0.0,0.0
1,0.890411,0.533333,0.0,1.0,0.173469,...,1.0,0.0,0.0,0.0,0.0


In [7]:
from models.tfmodel import TFModelAdult
model = TFModelAdult(dataset)

[WARNING] From C:\Users\ignacy\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor [deprecation.py new_func]


DICE FOR NN

In [51]:
factual = dataset.df_test.iloc[5:6]

hyperparams = {"num": 20}
# load a recourse model and pass black box model
gs = Dice(model, hyperparams)

# generate counterfactual examples
counterfactuals = gs.get_counterfactuals(factual)
counterfactuals

,age,education.num,capital.gain,capital.loss,hours.per.week,...,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,0.479452,0.333333,1.0,0.0,0.244898,...,1.0,0.0,0.0,0.0,0.0
1,0.479452,0.333333,0.3,0.0,0.244898,...,1.0,0.0,0.0,0.0,0.0
2,0.479452,0.333333,0.7,0.0,0.244898,...,1.0,0.0,0.0,0.0,0.0
3,0.479452,0.333333,0.8,0.0,0.244898,...,1.0,0.0,0.0,0.0,0.0
4,0.479452,0.333333,0.6,0.0,0.244898,...,1.0,0.0,0.0,0.0,0.0
5,0.479452,0.333333,1.0,0.0,0.244898,...,1.0,0.0,0.0,0.0,0.0
6,0.479452,0.333333,0.7,0.0,0.244898,...,1.0,0.0,0.0,0.0,1.0
7,0.479452,0.333333,0.6,0.0,0.244898,...,1.0,0.0,1.0,0.0,0.0
8,0.479452,0.333333,0.8,0.0,0.244898,...,1.0,0.0,0.0,0.0,0.0
9,0.479452,0.333333,0.6,0.0,0.244898,...,1.0,0.0,0.0,0.0,0.0


In [33]:
display_results(model, factual, counterfactuals)

Factual class: [[1. 0.]]
Counterfactuals class:
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]


CAUSAL RECOURSE

In [24]:
from carla.recourse_methods.catalog.causal_recourse import samplers, constraints
hypers = {
    'optimization_approach': "brute_force",
    'num_samples': 10,
    'scm': None,
    "constraint_handle": constraints.point_constraint,
    "sampler_handle": samplers.sample_true_m0,
}


gs = CausalRecourse(model, hypers)
# generate counterfactual examples
counterfactuals = gs.get_counterfactuals(dataset.df[:1])
counterfactuals

KeyboardInterrupt: 

GROWING SPHERES

In [45]:
gs = GrowingSpheres(model)
# generate counterfactual examples
counterfactuals = gs.get_counterfactuals(factual)
counterfactuals

,age,education.num,capital.gain,capital.loss,hours.per.week,...,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,0.479452,0.588754,1.351067,-0.284425,0.65422,...,0.0,0.0,1.0,1.0,1.0


In [35]:
display_results(model, factual, counterfactuals)

Factual class: [[1. 0.]]
Counterfactuals class:
[0. 1.]


ACTIONABLE RECOURSE (AR)

In [53]:
hyperparams = {
    'fs_size': 100
}

ar = ActionableRecourse(model, hyperparams)
counterfactuals = ar.get_counterfactuals(factual)
counterfactuals

[INFO] Start generating LIME coefficients [model.py get_counterfactuals]
[INFO] Finished generating LIME coefficients [model.py get_counterfactuals]
obtained 100 items in 1.2 seconds


,age,education.num,capital.gain,capital.loss,hours.per.week,...,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


In [37]:
display_results(model, factual, counterfactuals)

Factual class: [[1. 0.]]
Counterfactuals class:
[0. 1.]


CONTRASTIVE EXPLANATIONS METHOD (CEM)

In [72]:
hyperparams = {
        "data_name": 'test1',
        "batch_size": 1,
        "kappa": 0.1,
        "init_learning_rate": 0.01,
        "binary_search_steps": 9,
        "max_iterations": 100,
        "initial_const": 10,
        "beta": 0.9,
        "gamma": 0.0,
        "mode": "PN",
        "num_classes": 2,
        "ae_params": {"hidden_layer": [20, 10, 7], "train_ae": True, "epochs": 5},
}


sess = None

cm = CEM(sess = sess, mlmodel=model, hyperparams=hyperparams)

counterfactuals = cm.get_counterfactuals(factual)
counterfactuals

Train on 22792 samples, validate on 9769 samples
Epoch 1/5
22792/22792 [==============================] - 1s 34us/step - loss: 0.0665 - val_loss: 0.0320
Epoch 2/5
22792/22792 [==============================] - 0s 20us/step - loss: 0.0293 - val_loss: 0.0288
Epoch 3/5
22792/22792 [==============================] - 0s 20us/step - loss: 0.0276 - val_loss: 0.0275
Epoch 4/5
22792/22792 [==============================] - 0s 20us/step - loss: 0.0262 - val_loss: 0.0257
Epoch 5/5
22792/22792 [==============================] - 0s 20us/step - loss: 0.0247 - val_loss: 0.0245


AttributeError: 'NoneType' object has no attribute 'run'

Feasible and Actionable Counterfactual Explanations (FACE)

In [80]:
hyperparams = {
    'mode': 'knn',
    #'fraction':,
}
fc = Face(model, hyperparams)

counterfactuals = fc.get_counterfactuals(factual)
counterfactuals


,age,education.num,capital.gain,capital.loss,hours.per.week,...,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,0.30137,0.6,0.086141,0.0,0.397959,...,1.0,0.0,0.0,0.0,0.0


In [81]:
display_results(model, factual, counterfactuals)

Factual class: [[1. 0.]]
Counterfactuals class:
[0.192 0.808]
